## 데이터 시각화
### 시각화를 위해 필요한 데이터 추출
>   - 위도/경도 데이터

In [2]:
# geocoder 2.0 API로 위도, 경도 데이터 찾기
import requests
from tqdm import tqdm
import pandas as pd
import pymysql

#DB 접속 정보
with open("./db.config",'r') as f:
    db_data = f.read()
db_data = eval(db_data)

#DB 접속
_host = db_data['DB_HOST']
_user = db_data['DB_USER']
_password = db_data['DB_PASSWORD']
_name = db_data['DB_NAME']

con = pymysql.connect(host=f'{_host}', user=f'{_user}', password=f'{_password}', db=f'{_name}', charset='utf8')
cur = con.cursor()

tmp = ''
data = []
failed = []

lh_df = pd.read_sql_query('select * from allLh', con=con)
ansim_df = pd.read_sql_query('select * from ansim', con=con)

for idx, row in tqdm(lh_df.iterrows()):
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": row['adresWay'],
        "format": "json",
        "type": "road",
        "key": "[API-KEY]"
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        tmp = response.json()
        try:
            data.append(tmp['response']['result']['point'])
        except Exception as e:
            params["type"] = "parcel"
            response = requests.get(apiurl, params=params)
            if response.status_code == 200:
                tmp = response.json()
                try:
                    data.append(tmp['response']['result']['point'])
                except Exception as e:
                    failed.append(idx)

for idx, row in tqdm(ansim_df.iterrows()):
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": row['adresWay'],
        "format": "json",
        "type": "road",
        "key": "[API-KEY]"
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        tmp = response.json()
        try:
            data.append(tmp['response']['result']['point'])
        except Exception as e:
            params["type"] = "parcel"
            response = requests.get(apiurl, params=params)
            if response.status_code == 200:
                tmp = response.json()
                try:
                    data.append(tmp['response']['result']['point'])
                except Exception as e:
                    failed.append(idx)
                    
# 위도 경도 데이터 저장
for i in range(len(lh_df)):
    ansim_df.loc[i, 'x'] = data[i]['x']
    ansim_df.loc[i, 'y'] = data[i]['y']

ansim_df.loc[14,'x']='126.8982447'
ansim_df.loc[14,'y']='37.5251247'
ansim_df.loc[41,'x']='126.9167221'
ansim_df.loc[41,'y']='37.5507977'
ansim_df.loc[27,'x']='126.837949'
ansim_df.loc[27,'y']='37.5406894'

/tmp/ipykernel_16706/3256404656.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lh_df = pd.read_sql_query('select * from allLh', con=con)
/tmp/ipykernel_16706/3256404656.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ansim_df = pd.read_sql_query('select * from ansim', con=con)
67it [00:12,  5.44it/s]


KeyboardInterrupt: 

In [ ]:
# CSV 파일 저장
lh_df.to_csv('lh_df.csv', index=False)
ansim_df.to_csv('ansim_df.csv', index=False)

### 청년 주택 정보 제공

In [5]:
where_lh = pd.read_csv("./data/lh_df.csv")
where_ansim = pd.read_csv("./data/ansim_df.csv")
#where_lh = lh_df.copy()``
#where_ansim = ansim_df.copy()

> - Marker 찍기
> - Popup 활성화

In [9]:
import folium
where_hang = where_lh[where_lh['kindRental'] != '매입임대']
where_all = pd.concat([where_hang,where_ansim])
popmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
target =  where_all[['x', 'y', 'kindRental','adresWay','adresGu','kindHouse','totArea','scaleTot']].copy()
target['color'] =target.kindRental.apply(lambda x : 'red' if x =="매입임대" else ('blue' if x == "행복주택"  else 'orange')   )
popmap = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
for idx, row in target.iterrows():
    folium.Marker([row.y, row.x],
                  popup=f"""<div style="white-space: nowrap;">주소: {row.adresWay}<br>주택 종류: {row.kindHouse}<br>면적: {row.totArea}<br>세대수: {row.scaleTot}</div>""",              
                  icon=folium.Icon(color=row.color, icon='star')
                 ).add_to(popmap)

In [ ]:
# 지도 저장
popmap.save("./final.html")